# Análise de acidentes fatais na estrada na Austrália entre 1989 e 2021


In [ ]:
#TODO DESCOMENTAR PARA ENTREGA
# pd.set_option('display.max_columns', None)
# path_to_arthurs_drive = '/content/drive/MyDrive/IA - P3/Crash_Data.csv'
# df = pd.read_csv(path_to_arthurs_drive)

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
# from sklearn.impute import KNNImputer
# from matplotlib.dates import DateFormatter

import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)

df = pd.read_csv("Crash_Data.csv")

# 1. Visualizando os dados brutos

In [ ]:
df.head(5)

In [ ]:
# Emite um relatório simples sobre TODAS AS COLUNAS do Data frame
for column in df:
  soma_valores_nulos = df[column].isnull().sum()
  total_valores = len(df[column])
  porcentagem_nulos = round((soma_valores_nulos / total_valores) * 100, 2)

  print(f'Coluna {column} possui {soma_valores_nulos} valores nulos do total de {total_valores} -> {porcentagem_nulos}% \n valores diferentes: {set(df[column])} \n\n')


# 2. Preparando dados para análise

In [3]:
# Separa as colunas que serão usadas para análise e descarta as demais
df = df[[
  'Time',
  'Time of day',
  'Day of week',
  'Month',
  'Year',
  'Dayweek',
  'Crash Type',
  'Road User',
  'Gender',
  'Age',
  'Age Group',
  'State',
  'Speed Limit',
  'Bus Involvement',
  'Articulated Truck Involvement',
  'Christmas Period',
  'Easter Period',
]]


In [6]:
df.head(5)

,Time,Time of day,Day of week,Month,Year,Dayweek,Crash Type,Road User,Gender,Age,Age Group,State,Speed Limit,Bus Involvement,Articulated Truck Involvement,Christmas Period,Easter Period
0,0:30,Night,Weekend,9,2021,Sunday,Single,Motorcycle rider,Male,38,26_to_39,Vic,NaN,NaN,NaN,No,No
1,23:31,Night,Weekend,9,2021,Saturday,Multiple,Pedestrian,Female,28,26_to_39,SA,110,No,No,No,No
2,23:00,Night,Weekend,9,2021,Saturday,Single,Passenger,Male,19,17_to_25,Vic,NaN,NaN,NaN,No,No
3,22:25,Night,Weekend,9,2021,Saturday,Single,Driver,Male,23,17_to_25,Vic,NaN,NaN,NaN,No,No
4,5:15,Night,Weekend,9,2021,Saturday,Single,Motorcycle rider,Male,46,40_to_64,Vic,NaN,NaN,NaN,No,No


In [4]:
# Drop nas linhas onde a existe ao menos um valor nulo
df = df.dropna()

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51997 entries, 1 to 52842
Data columns (total 17 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   Time                           51997 non-null  object
 1   Time of day                    51997 non-null  object
 2   Day of week                    51997 non-null  object
 3   Month                          51997 non-null  int64 
 4   Year                           51997 non-null  int64 
 5   Dayweek                        51997 non-null  object
 6   Crash Type                     51997 non-null  object
 7   Road User                      51997 non-null  object
 8   Gender                         51997 non-null  object
 9   Age                            51997 non-null  int64 
 10  Age Group                      51997 non-null  object
 11  State                          51997 non-null  object
 12  Speed Limit                    51997 non-null  object
 13  Bus In

# 3. Analise gráfica dos dados já tratados


<!-- Adicionar alguns gráficos para analisarmos os dados já tratados e entender melhor os possíveis insights a serem extraídos (classificações e regressões) -->

In [ ]:
# Gráfico

In [1]:
# Gráfico

In [ ]:
# Gráfico

# 4. Criando o nosso modelo de Machine Learning

Nessa etapa nós:
- Separamos os dados de treino e teste
- Aplicamos o one-hot encoding ou o label encoding para o algoritmo conseguir processar
- Aplicamos o algoritmo de regressão linear ou de classificação
- Utilizamos das métricas de regressão e métricas de classificação, para *avaliar* a regressão/classificação feita.

## 4.1. Separando os dados de treino e teste

Nessa etapa podemos usar da análise gráfica feita anteriormente para entender quais as possíveis previsões e classificações podemos aplicar em novos dados, com base nos dados históricos (dataset).

Como possíveis previsões e classificações a serem feitas, temos:

*Previsões com regressão linear*
- Idade / idade média
- Limite de velocidade
- Quantidade de acidentes em feriados
  
*Classificações*
- Dias da semana
- Gênero da vítima
- Faixa etária

In [5]:
# Testando o split do df para prever o dia da semana

# Separando os dados para CLASSIFICAR o dia da semana

# Axis 0 -> linhas   | Axis 1 -> colunas
X = df.drop(['Dayweek'], axis=1)
y = df['Dayweek']

X

,Time,Time of day,Day of week,Month,Year,Crash Type,Road User,Gender,Age,Age Group,State,Speed Limit,Bus Involvement,Articulated Truck Involvement,Christmas Period,Easter Period
1,23:31,Night,Weekend,9,2021,Multiple,Pedestrian,Female,28,26_to_39,SA,110,No,No,No,No
5,4:00,Night,Weekend,9,2021,Multiple,Motorcycle rider,Male,19,17_to_25,Qld,100,No,No,No,No
6,19:10,Night,Weekend,9,2021,Single,Driver,Female,20,17_to_25,WA,100,No,No,No,No
9,2:00,Night,Weekend,9,2021,Single,Driver,Male,47,40_to_64,Qld,60,No,No,No,No
10,21:00,Night,Weekday,9,2021,Single,Driver,Male,24,17_to_25,SA,80,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52838,17:05,Day,Weekday,1,1989,Single,Passenger,Female,5,0_to_16,NSW,60,Yes,No,No,No
52839,6:00,Day,Weekday,1,1989,Single,Passenger,Male,19,17_to_25,WA,110,No,No,Yes,No
52840,6:00,Day,Weekday,1,1989,Single,Passenger,Male,17,17_to_25,WA,110,No,No,Yes,No
52841,6:00,Day,Weekday,1,1989,Single,Passenger,Male,17,17_to_25,WA,110,No,No,Yes,No


In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [9]:
from category_encoders import OrdinalEncoder
# A razão é que a maioria dos algoritmos de Machine Learning trabalham com features/variáveis em forma numérica. 
# A maioria das variáveis categóricas que vimos são do tipo texto/string. 
# Elas podem ser convertidas para inteiros ou floats. 
# Esse processo de converter uma variável textual para um inteiro é chamado de encoding (codificação).


# Label encoding ou one-hot encoding

# Para saber qual codificação utilizar, é necessário entender a diferença entre:

# - variáveis categóricas * ordinais *
# - variáveis categóricas * nominais *

# categóricas * ordinais *
# onde existe uma ordem intrínseca entre as categorias (“pequeno”, “médio” e “grande”)

# categóricas * nominais *
# onde não há ordem ou hierarquia ("")

ordinal_encoder = OrdinalEncoder(cols=['Road User'])

ordinal_encoder.fit(X_train)

X_train_encoded = ordinal_encoder.transform(X_train)
X_test_encoded = ordinal_encoder.transform(X_test)

X_train_encoded

,Time,Time of day,Day of week,Month,Year,Crash Type,Road User,Gender,Age,Age Group,State,Speed Limit,Bus Involvement,Articulated Truck Involvement,Christmas Period,Easter Period
3927,8:00,Day,Weekday,4,2018,Multiple,1,Female,63,40_to_64,Qld,100,No,No,No,No
48876,11:15,Day,Weekday,7,1990,Single,2,Male,67,65_to_74,NSW,60,No,No,No,No
17921,17:00,Day,Weekday,7,2007,Single,1,Female,41,40_to_64,Qld,60,No,No,No,No
6092,15:00,Day,Weekday,7,2016,Multiple,1,Male,23,17_to_25,Qld,80,No,Yes,No,No
52337,16:30,Day,Weekday,3,1989,Multiple,1,Male,70,65_to_74,WA,-9,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11424,21:10,Night,Weekend,2,2012,Single,1,Male,61,40_to_64,WA,50,No,No,No,No
45558,13:10,Day,Weekday,1,1992,Multiple,4,Female,68,65_to_74,SA,60,No,No,No,No
38966,5:50,Night,Weekend,5,1995,Multiple,5,Male,28,26_to_39,SA,100,No,No,No,No
907,22:55,Night,Weekday,12,2020,Single,4,Male,24,17_to_25,NT,60,No,No,No,No


In [10]:
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(random_state=42)
classifier.fit(X_train_encoded, y_train)

ValueError: could not convert string to float: '8:00'

# Características da Análise para explicação na documentação/apresentação
#### 1. Aprendizado: Supervisionado
#### 2. Classificação: Multiclass Classification 